In [2]:
import lightgbm as lgb
import numpy as np
import pandas as pd

In [5]:
df = pd.DataFrame()
leng = 10000
df['f1'] = np.concatenate([np.random.normal(0.2, 0.5, leng), np.random.normal(1, 0.5, leng)])
df['f2'] = np.concatenate([np.random.normal(0.2, 0.5, leng), np.random.normal(1, 0.5, leng)])
df['f3'] = np.concatenate([np.random.normal(0.2, 0.5, leng), np.random.normal(1, 0.5, leng)])
df['f4'] = np.concatenate([np.random.normal(0.2, 0.5, leng), np.random.normal(1, 0.5, leng)])
df['label'] = np.concatenate([ np.zeros(leng), np.ones(leng),])

dff = df.sample(len(df))
ratio =0.8
length = int(len(dff) * ratio)
trainset = dff.iloc[length:]
valset = dff.iloc[:length]

In [6]:
y_train = trainset['label'].values
y_val = valset['label'].values

lgb_train = lgb.Dataset(trainset[['f1', 'f2', 'f3', 'f4']], y_train)
lgb_val = lgb.Dataset(valset[['f1', 'f2', 'f3', 'f4']], y_val )

In [7]:
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 2000,
        'learning_rate': 0.1,
        'num_leaves': 3,
        'num_threads': 6, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 390,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 0.7, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'early_stopping_round':100,
        'bagging_fraction': 0.7, #Randomly select part of data without resampling
        'bagging_freq': 1, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        #'max_bin': 255,
        'verbose': 0,
        'scale_pos_weight': 400,
        'metric' : [ 'auc']
    }

model = lgb.train(params, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[10]	valid_0's auc: 0.918942
[20]	valid_0's auc: 0.926289
[30]	valid_0's auc: 0.933644
[40]	valid_0's auc: 0.957821
[50]	valid_0's auc: 0.962787
[60]	valid_0's auc: 0.969829
[70]	valid_0's auc: 0.977126
[80]	valid_0's auc: 0.980102
[90]	valid_0's auc: 0.98179
[100]	valid_0's auc: 0.983122
[110]	valid_0's auc: 0.984093
[120]	valid_0's auc: 0.984388
[130]	valid_0's auc: 0.984927
[140]	valid_0's auc: 0.984974
[150]	valid_0's auc: 0.98509
[160]	valid_0's auc: 0.985229
[170]	valid_0's auc: 0.985209
[180]	valid_0's auc: 0.9854
[190]	valid_0's auc: 0.985542
[200]	valid_0's auc: 0.985512
[210]	valid_0's auc: 0.985507
[220]	valid_0's auc: 0.985599
[230]	valid_0's auc: 0.985628
[240]	valid_0's auc: 0.985578
[250]	valid_0's auc: 0.98564
[260]	valid_0's auc: 0.985662
[270]	valid_0's auc: 0.985556
[280]	valid_0's auc: 0.985645
[290]	valid_0's auc: 0.98557
[300]	valid_0's auc: 0.985544
[310]	valid_0's auc: 0.985541
[320]	valid_0's auc: 0

In [8]:
model.best_iteration

256